In [0]:
RAW_PATH = "/Volumes/bronze_dev/global_mart_retail/raw_data"
BRONZE_DB = "bronze_dev"
TABLE = "raw_data"
FORMAT = "delta"


from pyspark.sql.functions import (
    current_timestamp, input_file_name, sha2, concat_ws
)

df = (
    spark.read
         .option("header", True)
         .option("inferSchema", False)
         .option("encoding", "UTF-8")
         .option("mode", "PERMISSIVE")
         .csv("dbfs:/Volumes/bronze_dev/global_mart_retail/raw_data/Sample - Superstore.csv")
)




In [0]:
(
    df.write
      .format("delta")
      .mode("append")
      .saveAsTable("bronze_dev.global_mart_retail.stg_raw_data")
)

